# Ejercicio 2: Top 10 arrival airports in 2013

# Team Science t-ish

## Importamos librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import bz2
import os

In [2]:
#Empleamos el fichero 'bookings.csv.bz2'

In [ ]:
df_bookings=pd.read_csv('bookings.csv.bz2', sep='^', nrows=10) #Comprobamos las 10 primeras lineas

In [ ]:
df_bookings

In [ ]:
len(df_bookings)

In [ ]:
df_bookings.head(5)

In [49]:
#Tomo una muestra de 10.000 filas para poder trabajar el ejercicio
df_bookings=pd.read_csv('bookings.csv.bz2', compression='bz2', sep='^', memory_map=True, error_bad_lines=False, nrows=10000)

In [ ]:
#Agrupamos por Aeropuertos / Cuantos booking tenemos y el nº de pasajeros, incluidos cancelaciones
Tabla_booking_pasajeros=df_bookings.groupby('arr_port').agg({'arr_port': lambda x: x.value_counts(),\
                                             'pax':'sum'})
Tabla_booking_pasajeros

### Abrirmos el fichero original por partes empleando la función "chunk"

In [50]:
with pd.read_csv('bookings.csv.bz2',  compression='bz2', sep='^', memory_map=True, error_bad_lines=False, chunksize=10) as reader:
    reader
    for chunk in reader:
        print(chunk)

AttributeError: __enter__

### Limpiamos columnas, N#A, aseguramos tipo de campo y que no tengamos string raros

In [ ]:
df_bookings=df_bookings.fillna({'pax':0})   #Rellenamos con 0 los campos  de pasajeros vacíos

In [ ]:
df_bookings['pax']=df_bookings['pax'].astype(int)  #Quitamos decimales, pero me encuentro N#A

In [ ]:
df_bookings['arr_port']=df_bookings_2_colm['arr_port'].str.strip() #Retiramos los espacios en blanco

In [ ]:
df_bookings['arr_port']=df_bookings_2_colm['arr_port'].str.upper() #Todas Mayúsculas

In [ ]:
Total_pasajeros=df_bookings['pax'].sum()
Total_pasajeros

In [ ]:
pasajeros_por_aeropuerto=df_bookings.groupby('arr_port').sum()

In [ ]:
pasajeros_por_aeropuerto.sort_values(by = "pax", ascending= False)[0:10]

### Debemos asegurar que el nombre del aeropuerto mantiene el formato de 3 Letras Mayúsculas

In [ ]:
df_bookings.head(2)

In [ ]:
#Compruebo la longitud del campo 'arr_port' --> Tiene 8 de longitud.
a=df_bookings['arr_port'].iloc[1]
len(a)
print(f"El aeropuerto es:xxxx{a}xxxx")

In [ ]:
df_bookings[['arr_port','pax']]

# Usamos la función "usecols" para cargar el fichero de una vez
### Quedándonos solo con las dos columneas que necesitamos

In [3]:
#LEEMOS SOLO LAS COLUMNAS arr_port y pax mediante la función rad_csv y el argumento "usecols"
df_bookings_2_colm = pd.read_csv("bookings.csv.bz2", compression= "bz2", sep = "^",\
                       error_bad_lines = False,\
                       memory_map = True,\
                       usecols = ["arr_port", "pax"])

In [43]:
df_bookings_2_colm.head(20)

,arr_port,pax,len
0,LHR,-1,3
1,CLT,1,3
2,CLT,1,3
3,SVO,1,3
4,SVO,1,3
5,LGA,1,3
6,LGA,1,3
7,SIN,2,3
8,SIN,2,3
9,SIN,2,3


In [26]:
df_bookings_2_colm=df_bookings_2_colm.fillna({'pax':0}) #Rellenamos con 0 pasajeros las filas vacías

In [27]:
df_bookings_2_colm['pax']=df_bookings_2_colm['pax'].astype(int) #Campo pasajeros a int

In [28]:
len(df_bookings_2_colm.groupby('arr_port').sum()) #Total de agrupaciones por aeropuerto sin asegurar nombres

2275

In [29]:
df_bookings_2_colm['arr_port']=df_bookings_2_colm['arr_port'].str.strip() #Retiramos los espacios en blanco

In [31]:
len(df_bookings_2_colm['arr_port'].iloc[0]) #Confirmo que la longitud ahora es de 3

3

In [32]:
df_bookings_2_colm['arr_port']=df_bookings_2_colm['arr_port'].str.upper()

In [33]:
len(df_bookings_2_colm.groupby('arr_port').sum())

2275

In [39]:
df_bookings_2_colm.groupby('arr_port').sum()

,pax
arr_port,
AAB,30
AAE,810
AAL,810
AAN,20
AAQ,1650
...,...
ZWE,30
ZWS,620
ZYL,1290
